In [2]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

import pandas as pd


In [33]:


# Load new contact data from a CSV file assuming it has  'Name' 'Email' and 'Company'  columns existing
contacts = pd.read_excel(r"C:path\contacts.xlsx")  # Replace with your actual CSV file path

# Load already reached contact data from a CSV file
try:
    reached = pd.read_csv(r"C:path\reached.csv")  # Assumes 'Name' and 'Email' columns exist
except FileNotFoundError:
    reached = pd.DataFrame(columns=['Name', 'Email','Company'])  # If the file doesn't exist, create an empty DataFrame
reached


# Your email account credentials
sender_email = "name@example.com"
password = "hlknm lfknew iod2lknw jnflk3ew"

# SMTP server details (use for Gmail, for example)
smtp_server = "smtp.gmail.com"
smtp_port = 856

# Compose the email template
subject = " Subject"
body_template = """Good morning {Name},

I'm .. and I am currently pursuing ... at ... . I am writing to express my interest in joining your team as ...., with availability ....

Please find attached my resume for your review. I would greatly appreciate the opportunity to discuss my application with you and look forward to hearing from you.

Kind regards,

"""
# Path to the file you want to attach
attachment_path = r"C:path\Resume.pdf" # Replace with the actual file path

# Replace with the actual company names and enail domain eg. "GMAIL" : "gmail"
companies = {
    "COMPANY1": "COMPANY1EmailDomain",
    "COMPANY2": "COMPANY2EmailDomain",
    "COMPANY3": "COMPANY3EmailDomain",
    "COMPANY4": "COMPANY4EmailDomain" 
}


In [ ]:
def create_email(row):
    # Retrieve company key and construct email domain
    if pd.notna(row['Email']):
        return row['Email']

    full_name = row['Name'].strip().split()
    first_name = full_name[0]  # First word is the first name
    last_name = "".join(full_name[1:])  # Concatenate all remaining words as last name
    
    # Retrieve company key from the dictionary
    company_key = companies.get(row["Company"], "unknown")
    proposed_email = f"{first_name.lower()}.{last_name.lower()}@{company_key}.com"
    return proposed_email

contacts["Email"] = contacts.apply(create_email, axis=1)
contacts.to_csv("updated_contacts.csv", index=False)

In [ ]:
# Connect to the SMTP server
try:
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()  # Secure the connection
    server.login(sender_email, password)

    # Iterate through the contact list and send emails
    for _, row in contacts.iterrows():
        recipient_name = row['Name']
        recipient_email = row['Email']

         # Check if the person is already in the 'reached.csv' file
        if not reached[(reached['Email'] == recipient_email)].empty:
            print(f"Skipping {recipient_name} ({recipient_email}) - already contacted.")
            continue
            
        personalized_body = body_template.format(Name=row['Name'], Company=row['Company'])
        
        # Create the email message
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient_email
        msg['Subject'] = subject
        msg.attach(MIMEText(personalized_body, 'plain'))
        
        # Attach the file
        with open(attachment_path, "rb") as attachment_file:
            attachment = MIMEApplication(attachment_file.read(), _subtype="pdf")
            attachment.add_header('Content-Disposition', 'attachment', filename="Resume.pdf")
            msg.attach(attachment)
        
        # Send the email
        server.sendmail(sender_email, recipient_email, msg.as_string())

        print(f"Email sent successfully to {row['Name']}")
        # Add the recipient to the 'reached.csv' file
        new_entry = pd.DataFrame({'Name': [recipient_name], 'Email': [recipient_email]})
        reached = pd.concat([reached, new_entry], ignore_index=True)
    
    # Save the updated 'reached.csv' file
    reached.to_csv('reached.csv', index=False)
except Exception as e:
    print(f"Failed to send email: {e}")
finally:
    # Close the server connection
    server.quit()